The Writer SDK has a few methods to interact with training data files stored with your account. This notebook will go over uploading, downloading, and deleting your JSONL files.

In [1]:
import os
from writer import Writer
from writer.models import operations, shared

First, we have to give the library our Writer API info. Create a file named `.env` in this directory with the lines 
```
WRITER_ORG_ID=YOUR_ORG_ID
WRITER_API_KEY=YOUR_API_KEY
```
or simply set the variables in the cell below.

In [2]:
from dotenv import load_dotenv

load_dotenv()
org_id = os.environ.get("WRITER_ORG_ID")
api_key = os.environ.get("WRITER_API_KEY")

writer = Writer(
    security=shared.Security(
        api_key=api_key
    ),
    organization_id=org_id
)

To get a list of all your files, send a `ListFilesRequest` with `files.list()`. If you haven't uploaded any files, the response will just be an empty list. 

In [3]:
req = operations.ListFilesRequest()

res = writer.files.list(req)
if res.model_files_response is not None:
    print(res.model_files_response)

ModelFilesResponse(files=[])


Of course, listing your files isn't very useful if you can't interact with them. Here's an example of uploading a very basic JSONL file and then retrieving its contents from Writer servers. For more information on using files for training data see [here](https://dev.writer.com/docs/prepare-training-data).

In [4]:
file = open("test.jsonl", "w")
file.write("{\"prompt\": \"Test prompt\", \"completion\": \"Test completion###\"}")
file.close()

file = open(file.name, "r")

req = operations.UploadFileRequest(
    upload_model_file_request=shared.UploadModelFileRequest(
        file=shared.UploadModelFileRequestFile(file.read(), file.name)
    )
)
res = writer.files.upload(req)
print(res)
# Note that the file id is required to access your files, not the name
# This means you can have multiple files share a name and access them individually
file_id = res.model_file.id
file.close()

req = operations.GetFileRequest(file_id)
res = writer.files.get(req)
print(res)

UploadFileResponse(content_type='application/json', status_code=200, fail_response=None, headers={'action-id': '[uploadFile][id=5c0cf7c0-ce24-47ac-9509-f7ccb96e76c9]', 'date': 'Tue, 25 Apr 2023 23:57:42 GMT', 'content-type': 'application/json', 'content-length': '157', 'x-envoy-upstream-service-time': '202', 'x-readme-id': '512012', 'x-readme-label': '512012', 'downstream-remote-address-without-port': '130.211.0.215', 'strict-transport-security': 'max-age=86400; includeSubDomains', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'x-frame-options': 'SAMEORIGIN', 'referrer-policy': 'same-origin', 'access-control-expose-headers': 'Authorization, X-AUTH-TOKEN, X-AUTH-REFRESH-TOKEN', 'content-security-policy': "object-src 'none' ; script-src 'self' 'unsafe-eval' 'unsafe-inline' ; connect-src 'self' wss://enterprise-api.writer.com ; frame-src 'self' enterprise-api.qordoba.local https://*.writer.com ; img-src 'self' ; style-src 'self' 'unsafe-inline' ; font-src 'self

Now if you list your files again, your uploaded file should show up

In [5]:
req = operations.ListFilesRequest()

res = writer.files.list(req)
if res.model_files_response is not None:
    print(res.model_files_response)

ModelFilesResponse(files=[ModelFile(created_at=datetime.datetime(2023, 4, 25, 23, 57, 42, 356741, tzinfo=tzutc()), format='jsonl', id='94d6c49d-575d-4966-aeda-68196b0f152b', name='test.jsonl', number_of_samples=1, size=61)])


Run this cell to cleanup test files:

In [6]:
if os.path.exists(file.name):
    os.remove(file.name)

req = operations.DeleteFileRequest(file_id)
res = writer.files.delete(req)
print(res)

DeleteFileResponse(content_type='application/json', status_code=500, delete_file_200_application_json_object=None, fail_response=FailResponse(extras=None, tpe='fail.internal', errors=[FailMessage(description='Unhandled exception', extras=None, key='fail.internal.generic')]), headers={'action-id': '[deleteFile][id=e744bb79-41e1-461f-b40a-b77dd7cb2620]', 'date': 'Tue, 25 Apr 2023 23:57:42 GMT', 'content-type': 'application/json', 'content-length': '130', 'x-envoy-upstream-service-time': '97', 'x-readme-id': '512012', 'x-readme-label': '512012', 'downstream-remote-address-without-port': '130.211.3.157', 'strict-transport-security': 'max-age=86400; includeSubDomains', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'x-frame-options': 'SAMEORIGIN', 'referrer-policy': 'same-origin', 'access-control-expose-headers': 'Authorization, X-AUTH-TOKEN, X-AUTH-REFRESH-TOKEN', 'content-security-policy': "object-src 'none' ; script-src 'self' 'unsafe-eval' 'unsafe-inline' ; co